## Data Collection - All other replays

---

1000 replays were collected from each of the following ranks:

- Bronze
- Silver
- Gold
- Platinum
- Diamond
- Champion
- Grand Champion/SSL

In [2]:
import requests
import json
import pandas as pd

import time

In [226]:
def get_replays(num_replays, params, auth_key, collected_replays = 0, id_list = []):
    
    time.sleep(0.25)
    res = requests.get('https://ballchasing.com/api/replays/',
                  headers={
                      'Authorization': auth_key},
                  params = params
                  )
    data = res.json()
    collected_replays += len(data['list'])
    print(collected_replays, num_replays)
    if collected_replays <= num_replays:
        for i in range(len(data['list'])):
            id_list.append(data['list'][i]['id'])
            replay_end_date = data['list'][-1]['date']
            params['replay-date-before'] = replay_end_date
            
                
        get_replays(num_replays = num_replays, 
                    params = params, 
                    auth_key = auth_key,
                    collected_replays = collected_replays,
                    id_list = id_list
                   )
                
    return id_list



def get_all_players(id_list, auth_key):
    '''
    This function differs from previous notebooks - iit does not filter by player name so it can collect data from all players in the game
    '''
    errors = 0
    
    res = requests.get(f'https://ballchasing.com/api/replays/{id_list[0]}',
                  headers={
                      'Authorization': auth_key}
                          )
    data = res.json()
    
    player_data = data['orange']['players'][0]['stats']
    player_data = dict(player_data, **player_data['core'])
    player_data = dict(player_data, **player_data['boost'])
    player_data = dict(player_data, **player_data['movement'])
    player_data = dict(player_data, **player_data['positioning'])
    player_data = dict(player_data, **player_data['demo'])
    del player_data['core']
    del player_data['boost']
    del player_data['movement']
    del player_data['positioning']
    del player_data['demo']

#     player_data['title'] = data['title']
    player_data['player_rank'] = ''
    
    df = pd.DataFrame(columns = player_data.keys())
    
    for i, s in enumerate(id_list):
        if i % 100 == 0:
            print(f"{i} id's have been checked, {df.shape[0]} replays collected")
        time.sleep(0.8)
        res = requests.get(f'https://ballchasing.com/api/replays/{s}',
                  headers={
                      'Authorization': auth_key}
                          )
        data = res.json()    
        if 'orange' not in data.keys():
            print(f"match {i} failed to retrieve data, {df.shape[0]} replays collected")
            errors += 1
            continue
        try:
            
            if len(data['orange']['players']) != 3 or len(data['blue']['players']) != 3:
                print(f"match {i} not 3V3")
                errors += 1
                continue
        except KeyError:
                errors += 1
                print(f"KeyError at match {i}, errors = {errors}")
                continue
        
        if df.shape[0] % 5 == 0:
            print(f"{df.shape[0]} games collected, {errors} errors")
                
        for j in range(len(data['orange']['players'])):
            try:
                
                player_data = data['orange']['players'][j]['stats']

                player_data = dict(player_data, **player_data['core'])
                player_data = dict(player_data, **player_data['boost'])
                player_data = dict(player_data, **player_data['movement'])
                player_data = dict(player_data, **player_data['positioning'])
                player_data = dict(player_data, **player_data['demo'])
                del player_data['core']
                del player_data['boost']
                del player_data['movement']
                del player_data['positioning']
                del player_data['demo']

                player_data['id'] = data['orange']['players'][j]['id']['id']
                player_data['player_rank'] = data['blue']['players'][j]['rank']['name']
                df.loc[player_data['id'], :] = player_data
            except KeyError:
                print(f"KeyError in id: {i} for orange player {j}")
                errors += 1

        for k in range(len(data['blue']['players'])):
            try:
                      
                player_data = data['blue']['players'][k]['stats']

                player_data = dict(player_data, **player_data['core'])
                player_data = dict(player_data, **player_data['boost'])
                player_data = dict(player_data, **player_data['movement'])
                player_data = dict(player_data, **player_data['positioning'])
                player_data = dict(player_data, **player_data['demo'])
                del player_data['core']
                del player_data['boost']
                del player_data['movement']
                del player_data['positioning']
                del player_data['demo']

                player_data['id'] = data['orange']['players'][k]['id']['id']
                player_data['player_rank'] = data['blue']['players'][j]['rank']['name']
                df.loc[player_data['id'], :] = player_data

            except KeyError:
                print(f"KeyError in id: {i} for blue player {k}")
                errors += 1
                
            

    return df, print(f"finished! {df.shape[0]} replays collected, errors = {errors}")

In [210]:
bronze_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'bronze-1',
                      'max-rank' : 'bronze-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500


In [200]:
silver_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'silver-1',
                      'max-rank' : 'silver-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500


In [203]:
gold_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'gold-1',
                      'max-rank' : 'gold-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500


In [204]:
platinum_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'platinum-1',
                      'max-rank' : 'platinum-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

diamond_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'diamond-1',
                      'max-rank' : 'diamond-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

champion_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'champion-1',
                      'max-rank' : 'champion-3'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

gc_ssl_ids = get_replays(num_replays=500, 
                      params = {
                      'playlist' : ['ranked-standard'],
                      'replay-date-after' : '2022-01-02T15:00:05+01:00',
                      'min-rank' : 'grand-champion'
                  }, 
                      auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5', id_list=[])

50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500
50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500
50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500
50 500
100 500
150 500
200 500
250 500
300 500
350 500
400 500
450 500
500 500
550 500


- IDs are appended into one list with all ranks, these are then run through the get_data function

In [211]:
len(set(bronze_ids)), len(set(silver_ids)), len(set(gold_ids)), len(set(platinum_ids)), len(set(diamond_ids)), len(set(champion_ids)), len(set(gc_ssl_ids)),   

(422, 371, 468, 491, 413, 477, 482)

In [212]:
ids = bronze_ids + silver_ids + gold_ids + platinum_ids + diamond_ids + champion_ids + gc_ssl_ids

In [216]:
ids = list(set(ids))

In [90]:
len(replay_id_list)

10000

In [230]:
replays = get_all_players(id_list = ids, 
             auth_key='mPuO0QZqG0wXdB9gDZHp7KKI09KplWLsYXkJQzJ5')

0 id's have been checked, 0 replays collected
0 games collected, 0 errors
KeyError in id: 2 for orange player 0
KeyError in id: 2 for orange player 1
15 games collected, 2 errors
KeyError in id: 5 for orange player 2
KeyError in id: 5 for blue player 0
KeyError in id: 5 for blue player 1
KeyError in id: 5 for blue player 2
20 games collected, 6 errors
KeyError in id: 10 for orange player 1
KeyError in id: 10 for orange player 2
KeyError in id: 10 for blue player 0
KeyError in id: 10 for blue player 1
KeyError in id: 10 for blue player 2
30 games collected, 11 errors
45 games collected, 11 errors
KeyError in id: 17 for orange player 0
KeyError in id: 17 for orange player 2
KeyError in id: 17 for blue player 0
KeyError in id: 17 for blue player 1
KeyError in id: 17 for blue player 2
KeyError in id: 18 for orange player 0
KeyError in id: 18 for orange player 1
KeyError in id: 19 for orange player 0
KeyError in id: 19 for orange player 1
KeyError in id: 19 for orange player 2
KeyError in i

In [233]:
replays = replays[0]

In [241]:
replays.player_rank

e8447c7649ce4c6cb88152d742689bd0           Silver II Division 2
TophTophRhodes                             Silver II Division 2
ff88618ff2010900                           Silver II Division 2
76561199080906162                         Platinum I Division 1
GonzoTacoBrown                            Silver III Division 4
                                               ...             
alonso_aco20                               Bronze II Division 4
KnownCart2595                              Bronze II Division 4
ccd73be9c9024cdeb7f266dae0511bf5    Grand Champion I Division 1
b9c630f7bf8444a096ef7f35743deb89    Grand Champion I Division 1
76561198973035879                   Grand Champion I Division 1
Name: player_rank, Length: 5134, dtype: object

In [242]:
replays.to_csv('../data/all_replays.csv')